# Exercise 5.2 EKF Localization (Range-Bearing)

## Theoretical background

The Kalman filter is one of the best studied techniques for filtering and prediction of linear systems.
Nevertheless, you should already know that our system of measurements (i.e. the observation function) and movement (i.e. pose composition) are non-linear.
Therefore, we will focus from the get-go in the Extended Kalman Filter, which is adapted to work with non-linear systems.

The EKF algorithm consists of 2 phases: **prediction** and **correction**.

$$
  \begin{aligned}
      \verb!def !& \verb!ExtendedKalmanFilter!(\mu_{t-1},\Sigma_{t-1}, u_t, z_t): \\
      & \textbf{Prediction.} \\
      & \bar\mu_t = g(\mu_{t-1}, u_t) = \mu_{t-1} \oplus u_t &\text{(1. Pose prediction)}\\
      & \bar\Sigma_t = G_t \Sigma_{t-1} G_t^T + R_t &\text{(2. Uncertainty of prediction)}\\
      & \textbf{Correction.} \\
      & K_t = \bar\Sigma_t H^T_t (H_t \bar\Sigma_t H^T_t + Q_t)^{-1} &\text{(3. Kalman gain)}\\
      & \mu_t = \bar\mu_t + K_t (z_t - h(\bar\mu_t)) &\text{(4. Pose estimation)}\\
      & \Sigma_t = (I - K_t H_t) \bar\Sigma_t &\text{(5. Uncertainty of estimation)}\\
      & \verb!return ! \mu_t, \Sigma_t
  \end{aligned}
$$

Please, notice that $R_t$ is the covariance of the motion $u_t$ in the coordinate system of the predicted pose $(\bar x_t)$, then (Note: $J_2$ is our popular Jacobian for the motion command, you could also use $J_1$):

$$R_t = J_2 \Sigma_{u_t} J_2^T \quad\text{with}\quad J_2 = \frac{\partial g(\mu_{t-1}, u_t)}{\partial u_t}$$

Where:

  - $(\mu_t, \Sigma_t)$ represents our robots pose.
  - $(u_t, \Sigma_{u_t})$ is the movement command received, and its respective uncertainty.
  - $(z_t, Q_t)$ are the observations taken, and their covariance.
  - $G_t$ and $H_t$ are the Jacobians of the motion model and the observation model respectively:

$$G_t = \frac{\partial g(\mu_{t-1}, u_t)}{\partial x_{t-1}}, \qquad H_t = \frac{\partial h(\bar\mu_t)}{\partial x_t}$$

In this exercise we are going to implement the EKF localization algorithm using a map of landmarks and a sensor providing range and bearing measurements from the robot pose to such landmarks. You can use the attached code to ease the programming task.

In [ ]:
# IMPORTS

import numpy as np
from numpy import random
from numpy import linalg
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt

from utils.AngleWrap import AngleWrapList
from utils.PlotEllipse import PlotEllipse
from utils.Drawings import DrawRobot, drawFOV, drawObservations
from utils.Jacobians import J1, J2
from utils.tcomp import tcomp

## 1. Getting an observation to a random landmark. 

Using the information provided by the `Map` in the code, complete the class method named `random_observation` that, given the robot pose, randomly selects a landmark and returns an observation from the range-bearing sensor using the `observe` method (that you have also to implement). Hint: use `randint(...)` from the scipy.random module.

In [ ]:
class Sensor():
    def __init__(self, cov):
        self.cov = cov
    
    def observe(self, from_pose, world, noisy=True):
        """Calculate observation relative to from_pose

        Args:
            from_pose: Position(real) of the robot which takes the observation
            world: List of world coordinates of some landmarks
            noisy: Flag, if true then add noise (Exercise 2)

        Returns:
                Numpy array of polar coordinates of landmarks from the perspective of our robot
                They are organised in a vertical vector ls = [d_0 , a_0, d_1, ..., a_n]'
                Dims (2*n_landmarks, 1) 
        """
        # TODO
        raise NotImplementedError
        
    def random_observation(self, from_pose, world, noisy=True):
        """ Get an observation from a random landmark 
            
            Args: Same as observe().
                
            Returns:
                z: Numpy array of obs. in polar coordinates
                landmark: Index of the randomly selected landmark in the world map
                    Although it is only one index, you should return it as
                    a numpy array.
        """
        # TODO
        raise NotImplementedError
        return z, landmark

## 2. Adding uncertainty to the sensor model

Modify the previous class to also consider the uncertainty in the sensor measurements defined by the matrix (`cov` in the code):

$$\Sigma_S = 
    \begin{bmatrix}
        \sigma^2_r & 0 \\
        0 & \sigma^2_{\varphi}
    \end{bmatrix}
$$

We only take uncertainty into account when we pass `noise=True` to our functions.

## 3. Simulating the robot motion. 
In the exercise 3.1 we commanded a mobile robot to follow a squared trajectory. Add random noise to each motion command (`noisy_u`) based on the following matrix, and update the true robot pose (`true_pose`):

$$\Sigma_{u_t} = 
    \begin{bmatrix}
        \sigma^2_{\Delta x} & 0 & 0\\
        0 & \sigma^2_{\Delta y} & 0\\
        0 & 0 & \sigma^2_{\Delta \theta}
    \end{bmatrix}
$$

In [ ]:
class Robot():
    def __init__(self, true_pose, cov_move):
        # Robot description (Starts as perfectly known)
        self.pose = true_pose
        self.true_pose = true_pose
        self.cov_move = cov_move

        # State -- Pose only
        self.xEst = true_pose
        self.PEst = np.zeros((3, 3))
        
    def step(self, u):
        self.pose = tcomp(self.pose,u) # New pose without noise
        noise = np.sqrt(self.cov_move)@random.randn(3,1) # Generate noise
        #TODO
        raise NotImplementedError
        noisy_u = None #  Apply noise to the control action
        self.true_pose = None  #  New noisy pose (real robot pose)
        
    def draw(self, fig, ax):
        DrawRobot(fig, ax, self.pose, color='r')
        DrawRobot(fig, ax, self.true_pose, color='b')
        DrawRobot(fig, ax, self.xEst, color='g')
        PlotEllipse(fig, ax, self.xEst, self.PEst, 4, color='g')

## 4. Fixing the robot pose according to the map.

Given that the position of the landmarks in the map is known, we can use this information in a Kalman filter, in our case an EKF. For that we need to implement the Jacobians of the observation model. Implement a function that, given the predicted pose in the first step of the Kalman filter, the selected landmark and the map, returns such Jacobian.

$$
\nabla h = \frac{\partial h}{\partial \{x,y,\theta \}} =
\begin{bmatrix}
    -\frac{x_i - x}{d} & - \frac{y_i - y}{d} & 0 \\
    \frac{y_i - y}{d^2} & -\frac{x_i - x}{d^2} & -1
\end{bmatrix}_{2 \times 3}
$$

In [ ]:
def getObsJac(xPred, lm, Map): 
    """ Obtain the Jacobian for all observations.

        Args:
            xPred: Position of our robot at which Jac is evaluated.
            lm: Numpy array of observations(indexes in map) in fov
            Map: Map containing the actual positions of the observations.

        Returns:
            jH: Jacobian matrix (2*n_landmaks, 3) 
    """
    # TODO 
    raise NotImplementedError

## 5. EKF filter

**Assignment**

Employing the previously coded functions, implement the EKF filter (both prediction and correction steps) and show the estimated pose and its uncertainty.

**Example**

The figure below shown an example of the execution of the EKF localization algorithm with the code implemented until this point.

<figure style="text-align:center">
  <img src="images/fig5-2-1.png" width="500" alt="">
  <figcaption>
      Fig. 1: Execution of the EKF algorithmn for localization, <br/>
      it shows the true (in blue) and expected (in red) poses, <br/>
      along the results from localization: pose and ellipse (in green), <br/>
      the existing landmarks (in cyan), <br/>
      and each observation made (dotted lines).
  </figcaption>
</figure>

In [ ]:
def EKFLocalization(robot, sensor, u, z, landmark, Map):
    """ Implement the EKF algorithm for localization
        
        Args:
            robot: Robot base(contains the state: xEst and PEst)
            sensor: Sensor of our robot.
            u: Movement command
            z: Observation received
            landmark: Index landmark observed in z
            Map
            
        Returns:
            xEst: New estimated pose
            PEst: Covariance of the estimated pose
    """
    # TODO Prediction 
    pass
    
    # TODO Correction (You need to compute the gain k and the innovation z-z_p) 
    pass

    raise NotImplementedError
    
    return xEst, PEst

Modify the main (`mode='one_landmark'`) to simulate that in each iteration the robot gathers an observation from the sensor (to a random landmark from the map). Draws a line from the robot to the landmark. Hint: you can use `plt.plot([x0, x1],[y0, y1])` for that.

After that can start testing our program.

In [ ]:
def main(robot,
         sensor,
         mode='one_landmark',
         nSteps=20, # Number of motions
         turning=5, # Number of motions before turning (square path)
         Size=50.0,
         NumLandmarks=10):
    seed = 1
    np.random.seed(seed)
    
    #Create map
    Map=Size*2*random.rand(2,NumLandmarks)-Size
    
    # MATPLOTLIB
    plt.ion()
    fig, ax = plt.subplots()
    plt.plot(Map[0,:],Map[1,:],'sc')
    plt.axis([-Size-5, Size+5, -Size-5, Size+5])
    
    robot.draw(fig, ax)
    fig.canvas.draw()
    
    # MAIN LOOP
    
    u = np.vstack([(2*Size-2*Size/3)/turning,0,0]) # control action
    plt.waitforbuttonpress(-1)    
    
    
    for k in range(0, nSteps-3): # Main loop
        u[2] = 0
        if k % turning == turning-1: # Turn?
            u[2] = -np.pi/2

        robot.step(u)
        
        # Get sensor observation/s
        if mode == 'one_landmark':
            # TODO (Exercise 5)
            raise NotImplementedError
        elif mode == 'one_landmark_in_fov':
            # TODO (Exercise 6)
            raise NotImplementedError
        elif mode == 'landmarks_in_fov':
            # TODO (Exercise 7)
            raise NotImplementedError
        
        robot.xEst, robot.PEst = EKFLocalization(robot, sensor, u, z, landmark, Map)

        # Drawings
        # Plot the FOV of the robot
        if mode == 'one_landmark_in_fov' or mode == 'landmarks_in_fov':
            h = sensor.draw(fig, ax, robot.true_pose)
        #end

        robot.draw(fig, ax)

        fig.canvas.draw()
        plt.waitforbuttonpress(-1)    

        if mode == 'one_landmark_in_fov' or mode == 'landmarks_in_fov':
            h.pop(0).remove()
        fig.canvas.draw()

In [ ]:
# RUN
mode = 'one_landmark'
#mode = 'one_landmark_in_fov'
#mode = 'landmarks_in_fov'
Size=50.0

# Robot base characterization
SigmaX = 0.8 # Standard deviation in the x axis
SigmaY = 0.8 # Standard deviation in the y axis
SigmaTheta = 0.1 # Bearing standar deviation
R = np.diag([SigmaX**2, SigmaY**2, SigmaTheta**2]) # Cov matrix

true_pose = np.vstack([-Size+Size/3, -Size+Size/3, np.pi/2])
robot = Robot(true_pose, R)

# Sensor characterization
SigmaR = 1 # Standard deviation of the range
SigmaB = 0.7 # Standard deviation of the bearing
Q = np.diag([SigmaR**2, SigmaB**2]) # Cov matrix

sensor = Sensor(Q)

#main(robot, sensor, mode=mode, Size=Size)

## 6. Modifying the sensor information.

Modifying the sensor information. Sensors exhibit certain physical limitations regarding their field of view and maximum operating distance (max. Range). Modify the code to consider that the sensor can only provide information from a random landmark in a limited range $r_l$ and a limited orientation $\pm \alpha$ with respect to the robot pose. That is the `'one_landmark_in_fov'` mode.

Complete the following class method. Then go back to the `one_landmark_in_fov` part in our `main()`. Use this new method to get all observations in the fov and then select a random one.

It could happen that any landmark exists in the field of view of the sensor, so the robot couldn’t gather sensory information in that iteration. Discuss how the uncertainty evolves.

In [ ]:
class FOVSensor(Sensor):
    def __init__(self, cov, fov, max_range):
        super().__init__(cov)
        self.fov = fov
        self.max_range = max_range
    
    def observe_in_fov(self, from_pose, world, noisy=True):
        """ Get all observations in the fov

        Args:
            from_pose: Position(real) of the robot which takes the observation
            world: List of world coordinates of some landmarks
            noisy: Flag, if true then add noise (Exercise 2)

        Returns:
            Numpy array of polar coordinates of landmarks from the perspective of our robot
            They are organised in a vertical vector ls = [d_0 , a_0, d_1, ..., a_n]'
            Dims (2*n_landmarks, 1) 
        """
        # TODO
        raise NotImplementedError
    
    def draw(self, fig, ax, from_pose):
        return drawFOV(fig, ax, from_pose, self.fov, self.max_range)
    
# RUN
mode = 'one_landmark_in_fov'
Size=50.0

# Robot base characterization
SigmaX = 0.8 # Standard deviation in the x axis
SigmaY = 0.8 # Standard deviation in the y axis
SigmaTheta = 0.1 # Bearing standar deviation
R = np.diag([SigmaX**2, SigmaY**2, SigmaTheta**2]) # Cov matrix

true_pose = np.vstack([-Size+Size/3, -Size+Size/3, np.pi/2])
robot = Robot(true_pose, R)

# Sensor characterization
SigmaR = 1 # Standard deviation of the range
SigmaB = 0.7 # Standard deviation of the bearing
Q = np.diag([SigmaR**2, SigmaB**2]) # Cov matrix

fov = np.pi/2 # field of view = 2*alpha
max_range = Size # maximum sensor measurement range

sensor = FOVSensor(Q, fov, max_range)

main(robot, sensor, mode=mode, Size=Size)

## 7. Adding more information from the sensor
Usually, sensors do not provide information from only a landmark. Complete the `lanmarks_in_fov` part of the `main()` to observe all the landmarks in fov. This also implies modifications in the functions for computing the Jacobian of the sensor model (it now has $2k$ rows and $3$ columns). I recommend creating a backup of this notebook before altering previous methods.

The figure below shows an example of the execution of EKF using information from all the landmarks within the FOV:

<figure style="text-align:center">
  <img src="images/fig5-2-2.png" width="500" alt="">
  <figcaption>
      Fig. 2: Execution of the EKF algorithmn for localization. <br/>
      Same as in Fig. 1, except now our robot can observe every <br/> 
      lanmark in its f.o.v. 
  </figcaption>
</figure>

In [ ]:
# RUN
mode = 'landmarks_in_fov'
Size=50.0

# Robot base characterization
SigmaX = 0.8 # Standard deviation in the x axis
SigmaY = 0.8 # Standard deviation in the y axis
SigmaTheta = 0.1 # Bearing standar deviation
R = np.diag([SigmaX**2, SigmaY**2, SigmaTheta**2]) # Cov matrix

true_pose = np.vstack([-Size+Size/3, -Size+Size/3, np.pi/2])
robot = Robot(true_pose, R)

# Sensor characterization
SigmaR = 1 # Standard deviation of the range
SigmaB = 0.7 # Standard deviation of the bearing
Q = np.diag([SigmaR**2, SigmaB**2]) # Cov matrix
fov = np.pi/2 # field of view = 2*alpha
max_range = Size # maximum sensor measurement range

sensor = FOVSensor(Q, fov, max_range)

main(robot, sensor, mode=mode, Size=Size)